In [251]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('fivethirtyeight')

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

## Load Data

In [140]:
data = pd.read_csv('train.csv')

In [141]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Explore Data

In [142]:
data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [143]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

### Check columns with missing values

In [205]:
missing = list(data.isnull().sum()[data.isnull().sum() > 0].index)
print(missing)

['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']


In [206]:
mis_cols_toberemoved = data.isnull().sum()[data.isnull().sum() > 0]/data.shape[0]
mis_cols_toberemoved = mis_cols_toberemoved > 0.5
mis_cols_toberemoved = list(mis_cols_toberemoved[mis_cols_toberemoved].index)
mis_cols_toberemoved

['Alley', 'PoolQC', 'Fence', 'MiscFeature']

### List of Columns with Categorical data

In [207]:
cat_cols = [col for col in data.columns if data[col].dtype == 'O' ]
print('Total Columns: {}'.format(len(data.columns)))
print('Categorical Columns: {}\n'.format(len(cat_cols)))
print(cat_cols)

Total Columns: 81
Categorical Columns: 43

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


### Check Cardinality of Categorical Columns & Remove columns with High Cardinality

In [208]:
print('Categorical Columns before Removal: {}'.format(len(cat_cols)))
card_col = [col for col in cat_cols if data[col].nunique()<10]
print('Categorical Columns after Removal: {}\n'.format(len(card_col)))
print(card_col)

Categorical Columns before Removal: 43
Categorical Columns after Removal: 40

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


### List Numerical Columns

In [209]:
num_cols = [col for col in data.columns if data[col].dtype in ('int64', 'float64') ]
num_cols.remove('Id')
print('Total Columns: {}'.format(len(data.columns)))
print('Numerical Columns: {}\n'.format(len(num_cols)))
print(num_cols)

Total Columns: 81
Numerical Columns: 37

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']


### Select Numerical Columns based on Correlation above 0.3

In [210]:
print('Numerical Columns before Removal: {}'.format(len(num_cols)))
num_cols = list(data[num_cols].corr()['SalePrice'][abs(data[num_cols].corr()['SalePrice'])>0.3].index)
print('Numerical Columns after Removal: {}\n'.format(len(num_cols)))
print(num_cols)

Numerical Columns before Removal: 37
Numerical Columns after Removal: 19

['LotFrontage', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'SalePrice']


In [211]:
## Lists to be used for Data Preparation
print(len(mis_cols_toberemoved))
print(len(missing))
print(len(card_col))
print(len(num_cols))

4
19
40
19


### Select Required Columns

In [212]:
card_col = [col for col in card_col if col not in mis_cols_toberemoved]
my_cols = card_col + num_cols
len(my_cols)

55

### Split Data into Features and Output

In [260]:
y = data[my_cols]['SalePrice']
X = data[my_cols].drop(['SalePrice'], axis=1)
print('Shape of Features Data {}'.format(X.shape))
print('Shape of Output Columns {}'.format(y.shape))

Shape of Features Data (1460, 54)
Shape of Output Columns (1460,)


### Train-Validation Split

In [261]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y,train_size=0.8, test_size=0.2,random_state=0)
print('Shape of Train Features Data {}'.format(X_train.shape))
print('Shape of Train Output Columns {}'.format(y_train.shape))
print('Shape of Validation Features Data {}'.format(X_valid.shape))
print('Shape of Validation Output Columns {}'.format(y_valid.shape))

Shape of Train Features Data (1168, 54)
Shape of Train Output Columns (1168,)
Shape of Validation Features Data (292, 54)
Shape of Validation Output Columns (292,)


### Create Pipeline for Imputation and OneHotEncoding

In [271]:
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_cols),
        ('cat', categorical_transformer, card_col)
    ])

In [272]:
# scores = -1 * cross_val_score(pip, X, y,
#                               cv=5,
#                               scoring='neg_mean_absolute_error')

# print("Average MAE score:", scores.mean())

### Function for Error Calculations

In [273]:
def errors(y_valid, X_valid):
    print('Mean Absulute Error (MAE):', mean_absolute_error(y_valid, pip.predict(X_valid)))
    print('Mean Absulute Percentage Error (MAPE):', mean_absolute_percentage_error(y_valid, pip.predict(X_valid)))
    print('Mean Squared Error (MSE):', mean_squared_error(y_valid, pip.predict(X_valid)))
    print('Root Mean Squared Error (RMSE):', mean_squared_error(y_valid, pip.predict(X_valid), squared=False))

### <font color='green'> 1. Model (Decesion Tree)

In [274]:
# Define model
model = RandomForestRegressor(n_estimators=500, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
pip = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

pip.fit(X_train, y_train)
errors(y_valid = y_valid, X_valid = X_valid)
# preds = pip.predict(X_valid)
# print('Mean Absulute Error (MAE):', mean_absolute_error(y_valid, preds))
# print('Mean Absulute Percentage Error (MAPE):', mean_absolute_error(y_valid, preds))
# print('Mean Squared Error (MSE):', mean_absolute_error(y_valid, preds))
# print('Root Mean Squared Error (RMSE):', mean_absolute_error(y_valid, preds))

ValueError: A given column is not a column of the dataframe